In [1]:
from tensorflow.keras.optimizers import Adam
from keras.utils import Sequence
import pandas as pd
import numpy as np
import os

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-06-30 17:08:23.870919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-30 17:08:24.772395: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataloaders.OptimizedDataGenerator_v2 import OptimizedDataGenerator
from models import *

In [3]:
model=CreateModel((16,16,20),n_filters=5,pool_size=3)
model.summary()

2025-06-30 17:08:26.466029: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 648 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b2:00.0, compute capability: 7.5


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16, 16, 20)]      0         
                                                                 
 q_separable_conv2d (QSepar  (None, 14, 14, 5)         285       
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 14, 14, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 14, 14, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 14, 14, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 4, 4, 5)           0     

In [4]:
# get best weights file
pitch = '50x12P5'
batch_size = 5000
fingerprint = '34c2da80'
timeslices = 20
files = os.listdir('/data/dajiang/smart-pixels/weights/dataset_3src_16x16_weights/weights-{}-bs{}-{}-{}t-nonquantized_input-checkpoints'.format(pitch, batch_size, fingerprint, timeslices))

vlosses = [float(f.split("-v")[1].split(".hdf5")[0]) for f in files]
bestfile = files[np.argmin(vlosses)]
model.load_weights('/data/dajiang/smart-pixels/weights/dataset_3src_16x16_weights/weights-{}-bs{}-{}-{}t-nonquantized_input-checkpoints/'.format(pitch, batch_size, fingerprint, timeslices)+bestfile)

print('Best model: {}'.format(bestfile))

Best model: weights.971-t-29391.59-v-30412.73.hdf5


In [5]:
# load in the test set
test_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = '/data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_{}_parquets/contained/TFR_files/{}t/TFR_val'.format(pitch, timeslices),
    quantize = True
)

Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/20t/TFR_val/metadata.json


In [6]:
# predicts test data
p_test = model.predict(test_generator)

complete_truth = None
for _, y in test_generator:
    if complete_truth is None:
        complete_truth = y
    else:
        complete_truth = np.concatenate((complete_truth, y), axis=0)

# creates df with all predicted values and matrix elements - 4 predictions, all 10 unique matrix elements
df = pd.DataFrame(p_test,columns=['x','M11','y','M22','cotA','M33','cotB','M44','M21','M31','M32','M41','M42','M43'])

# stores all true values in same matrix as xtrue, ytrue, etc.
df['xtrue'] = complete_truth[:,0]
df['ytrue'] = complete_truth[:,1]
df['cotAtrue'] = complete_truth[:,2]
df['cotBtrue'] = complete_truth[:,3]
df['M11'] = minval+tf.math.maximum(df['M11'], 0)
df['M22'] = minval+tf.math.maximum(df['M22'], 0)
df['M33'] = minval+tf.math.maximum(df['M33'], 0)
df['M44'] = minval+tf.math.maximum(df['M44'], 0)

df['sigmax'] = abs(df['M11'])
df['sigmay'] = np.sqrt(df['M21']**2 + df['M22']**2)
df['sigmacotA'] = np.sqrt(df['M31']**2+df['M32']**2+df['M33']**2)
df['sigmacotB'] = np.sqrt(df['M41']**2+df['M42']**2+df['M43']**2+df['M44']**2)

# calculates residuals for x, y, cotA, cotB
df['residualsX'] = df['xtrue'] - df['x']
df['residualsY'] = df['ytrue'] - df['y']
df['residualsA'] = df['cotAtrue'] - df['cotA']
df['residualsB'] = df['cotBtrue'] - df['cotB']

# calculates pulls for x, y, cotA, cotB
df['pullx'] = (df['xtrue']-df['x'])/df['sigmax']
df['pully'] = (df['ytrue']-df['y'])/df['sigmay']
df['pullcotA'] = (df['cotAtrue']-df['cotA'])/df['sigmacotA']
df['pullcotB'] = (df['cotBtrue']-df['cotB'])/df['sigmacotB']

# stores results as parquet
df.to_parquet("/home/dajiang/smart-pixels-ml/processed_parquets/dataset_3src_16x16/test-dataset_3src_16x16/{}_bs{}_{}_{}t_train-dataset_3src_16x16_test-dataset_3src_16x16_nonquantized-input_vars.parquet".format(pitch, batch_size, fingerprint, timeslices))

2025-06-30 17:08:39.954991: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906
2025-06-30 17:08:40.062794: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 1/21 [>.............................] - ETA: 21s

2025-06-30 17:08:40.157259: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 90.77MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-06-30 17:08:40.160107: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 242.99MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-06-30 17:08:40.162204: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 235.95MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-06-30 17:08:40.178265: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 
Traceback (most recent call last):

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/data_adapter.py", line 1064, in generator_fn
    yield x[i